In [1]:
import os
import time
import threading

#### Current Thread

In [10]:
print('Process Begin')
print(f'Process id: {os.getpid()}')
print(f'Current Thread name: {threading.current_thread().name}')
print('Sleeping')
time.sleep(5)
threading.current_thread().name = 'MyThread'
print(f'Current Thread name: {threading.current_thread().name}')
print('Process Exit')

Process Begin
Process id: 5427
Current Thread name: MyThread
Sleeping
Current Thread name: MyThread
Process Exit


#### Create a new Thread
1. Create an instance of Thread class
2. Write a sub-class of Thread and instantiate the sub-class

In [14]:
def run():
    for i in range(1,6):
        print(f'{i} --> Thread name: {threading.current_thread().name}, Process id: {os.getpid()}')
        time.sleep(1)

In [12]:
t = threading.Thread(target=run)
t.name='MyThread'
t.start()

1 --> Thread name: MyThread
2 --> Thread name: MyThread
3 --> Thread name: MyThread
4 --> Thread name: MyThread
5 --> Thread name: MyThread


In [28]:
print(f'Main Thread enter, Process id: {os.getpid()}')
t = threading.Thread(target=run, name='Thread-01')
t.start()
print(f't is alive: {t.is_alive()}')
t.join() # current thread (main thread) goes to wait state, gets notified when 't' is dead
print(f't is alive: {t.is_alive()}')
print(f'Main Thread exit')

Main Thread enter, Process id: 5534
1 --> Thread name: Thread-01, Process id: 5534
t is alive: True
2 --> Thread name: Thread-01, Process id: 5534
3 --> Thread name: Thread-01, Process id: 5534
4 --> Thread name: Thread-01, Process id: 5534
5 --> Thread name: Thread-01, Process id: 5534
t is alive: False
Main Thread exit


In [29]:
print(f'Main Thread enter, Process id: {os.getpid()}')

t1 = threading.Thread(target=run, name='Thread-01')
t2 = threading.Thread(target=run, name='Thread-02')

t1.start()
t2.start()

t1.join()
t2.join()

print(f'Main Thread exit')

Main Thread enter, Process id: 5534
1 --> Thread name: Thread-01, Process id: 5534
1 --> Thread name: Thread-02, Process id: 5534
2 --> Thread name: Thread-01, Process id: 5534
2 --> Thread name: Thread-02, Process id: 5534
3 --> Thread name: Thread-01, Process id: 5534
3 --> Thread name: Thread-02, Process id: 5534
4 --> Thread name: Thread-01, Process id: 5534
4 --> Thread name: Thread-02, Process id: 5534
5 --> Thread name: Thread-01, Process id: 5534
5 --> Thread name: Thread-02, Process id: 5534
Main Thread exit


#### Create a thread whose target function accepts numbers, adds them and display the sum

In [32]:
def add(*args):
    print(f'Thread name: {threading.current_thread().name}, Sum = {sum(args)}')

In [36]:
t1 = threading.Thread(target=add, name='First', args=(1,2,3,4,5))
t2 = threading.Thread(target=add, name='Second', args=(11,22,33))

t1.start()
t2.start()

Thread name: First, Sum = 15
Thread name: Second, Sum = 66


#### Create a Thread by Interitance
1. Write a sub-class of Thread class
2. Target method by default would be the instance method 'run' hence we override the run method
3. t.run is a way to specify the target in case we don't target to be run method
4. arguments can be set explicitly through 't.args' attribute and access as 'self.args' in the thread sub-class

In [10]:
class MyThread(threading.Thread):
    def run(self):
        for i in range(1,6):
            print(f'{i} --> Thread name: {threading.current_thread().name}, Process id: {os.getpid()}')
            time.sleep(1)

t1 = MyThread()
t1.name = 'MyThread-01'
t1.start()            

1 --> Thread name: MyThread-01, Process id: 5711
2 --> Thread name: MyThread-01, Process id: 5711
3 --> Thread name: MyThread-01, Process id: 5711
4 --> Thread name: MyThread-01, Process id: 5711
5 --> Thread name: MyThread-01, Process id: 5711


In [11]:
class MyThread(threading.Thread):
    def do_something(self):
        for i in range(1,6):
            print(f'{i} --> Thread name: {threading.current_thread().name}, Process id: {os.getpid()}')
            time.sleep(1)

t2 = MyThread()
t2.name = 'MyThread-02'
t2.run = t2.do_something # Explicitly set the target (Refer to point 3 above)
t2.start() 

1 --> Thread name: MyThread-02, Process id: 5711
2 --> Thread name: MyThread-02, Process id: 5711
3 --> Thread name: MyThread-02, Process id: 5711
4 --> Thread name: MyThread-02, Process id: 5711
5 --> Thread name: MyThread-02, Process id: 5711


#### Write a sub-class of Thread named AddThread
- the target method sums the arguments and display the sum along with thread name 

In [12]:
class AddThread(threading.Thread):
    def run(self):
        print(f'Thread name: {threading.current_thread().name}, Sum = {sum(self.args)}')

In [14]:
t1 = AddThread()
t1.name = 'First'
t1.args = (1,2,3,4,5) # Refer to point 4 above

t2 = AddThread()
t2.name = 'Second'
t2.args = (10,20,30) # Refer to point 4 above

t1.start()
t2.start()

Thread name: First, Sum = 15
Thread name: Second, Sum = 60


#### Thread assignment
- Write a class named Resource (not a sub-class of Thread)
- It will be have an instance field named 'data' (initialize it to 0 in the constructor)
- It will have an instance method named 'do_something'
- The method will increment data by one and display the name of the current thread followed by value of data
- The method will be target method for a thread

In [36]:
# Non Thread-safe Resource
class Resource:
    def __init__(self):
        self.data = 0
    
    def do_something(self):
        self.data += 1
        for i in range(1000000):
            pass
        print(f'{threading.current_thread().name} -- {self.data}')

In [38]:
# Thread-safe Resource
class Resource:
    def __init__(self):
        self.data = 0
        self.lock = threading.Lock() # only one lock per resource
    
    def do_something(self):
        self.lock.acquire()
        self.data += 1
        for i in range(1000000):
            pass
        print(f'{threading.current_thread().name} -- {self.data}')
        self.lock.release()

In [41]:
# Thread-safe Resource using with statement
class Resource:
    def __init__(self):
        self.data = 0
        self.lock = threading.Lock() # only one lock per resource
    
    def do_something(self):
        with self.lock:
            self.data += 1
            for i in range(1000000):
                pass
            print(f'{threading.current_thread().name} -- {self.data}')

In [42]:
r = Resource()

t1 = threading.Thread(target = r.do_something, name='First')
t2 = threading.Thread(target = r.do_something, name='Second')
t3 = threading.Thread(target = r.do_something, name='Third')

t1.start()
t2.start()
t3.start()

First -- 1
Second -- 2
Third -- 3


In [59]:
r = Resource()

for i in range(1,11):
    t = threading.Thread(target = r.do_something, name=f'Thread-{i}')
    t.start()
    t.join()

Thread-1 -- 1
Thread-2 -- 2
Thread-3 -- 3
Thread-4 -- 4
Thread-5 -- 5
Thread-6 -- 6
Thread-7 -- 7
Thread-8 -- 8
Thread-9 -- 9
Thread-10 -- 10


#### MultiProcessing in Python
- MultiThreading - Concurrent processing (uses 1 CPU which is shared by all the Threads) 
- MultiProcessing - Parallel processing (uses as many CPUs as available)

In [43]:
import os
import multiprocessing

print(f'Available processors: {os.cpu_count()}')

Available processors: 4


In [46]:
def worker1():
    print(f'In Worker-1 process, Process id: {os.getpid()}')
    
def worker2():
    print(f'In Worker-2 process, Process id: {os.getpid()}')

print(f'Main enter, Process id: {os.getpid()}')
    
p1 = multiprocessing.Process(target=worker1)
p2 = multiprocessing.Process(target=worker2)

p1.start()
p2.start()

print('\nBefore join')
print(f'Worker-1 is alive - {p1.is_alive()}')
print(f'Worker-2 is alive - {p2.is_alive()}')

p1.join()
p2.join()

print('\nAfter join')
print(f'Worker-1 is alive - {p1.is_alive()}')
print(f'Worker-2 is alive - {p2.is_alive()}')

print(f'\nMain exit')    

Main enter, Process id: 5711
In Worker-2 process, Process id: 6394
In Worker-1 process, Process id: 6392

Before join
Worker-1 is alive - True
Worker-2 is alive - True

After join
Worker-1 is alive - False
Worker-2 is alive - False

Main exit


#### Create 2 process objects
- One will square a given number (as parameter)
- The other will cube a given number (as parameter)
- Parameter are passed as args (similar to thread target function arguments)
- Make sure that only one process prints to the standard output at any given time

In [57]:
import os
import time
import multiprocessing

def square(lock,num):
    lock.acquire()
    print(f'Square process id: {os.getpid()}')
    print(f'Square of {num} is {num ** 2}')
    lock.release()
    time.sleep(20)

def cube(lock,num):
    with lock:
        print(f'Cube process id: {os.getpid()}')
        print(f'Cube of {num} is {num ** 3}')
    time.sleep(20)
    
print(f'Main process enter, Process id: {os.getpid()}')
lock = multiprocessing.Lock()

p1 = multiprocessing.Process(target=square, args=(lock,10))
p2 = multiprocessing.Process(target=cube, args=(lock,10))

p1.start()
p2.start()

p1.join()
p2.join()

print(f'Main process exit')

Main process enter, Process id: 5711
Square process id: 6683
Square of 10 is 100
Cube process id: 6686
Cube of 10 is 1000
Main process exit


#### Sharing data in MultiProcessing through a Queue

In [62]:
from multiprocessing import Process, Queue

#Child process code
def func(queue):
    value = queue.get()
    print(f'Child got: {value}')
    for i in range(1,value+1):
        queue.put(i ** 2)

#------------------------------------------

#Parent process code
q = Queue()
p = Process(target=func, args=(q,))
num = 5
q.put(num)
print(f'Parent sent: {num}')
p.start()
p.join()
print('\nIn Parent')
while not q.empty():
    print(q.get())

Parent sent: 5
Child got: 5

In Parent
1
4
9
16
25
